In [2]:
#model轉onnx(C2FViT)
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
import numpy as np
from C2FViT_model import C2F_ViT_stage, AffineCOMTransform, CustomAffineCOMTransform, Center_of_mass_initial_pairwise, CustomCenter_of_mass_initial_pairwise
from Functions import min_max_norm, pad_to_shape

class FullModel(nn.Module):
    def __init__(self, model, affine_transform, init_center):
        super(FullModel, self).__init__()
        self.model = model
        self.affine_transform = affine_transform
        self.init_center = init_center

    def forward(self, moving_img, fixed_img):
        # Center of mass initialization
        moving_img, init_flow = self.init_center(moving_img, fixed_img)
        
        # Downsample the images
        X_down = F.interpolate(moving_img, scale_factor=0.5, mode="trilinear", align_corners=True)
        Y_down = F.interpolate(fixed_img, scale_factor=0.5, mode="trilinear", align_corners=True)
        
        # Run the core model
        warpped_x_list, y_list, affine_para_list = self.model(X_down, Y_down)
        #rigid
        #affine_para_list[-1][0, 11] = 0
        #affine_para_list[-1][0, 10] = 0
        #affine_para_list[-1][0, 9] = 0
        #affine_para_list[-1][0, 8] = 0
        #affine_para_list[-1][0, 7] = 0
        #affine_para_list[-1][0, 6] = 0
        # Apply the affine transformation
        X_Y, affine_matrix = self.affine_transform(moving_img, affine_para_list[-1])
        
        return X_Y, affine_matrix, init_flow

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義模型
model = C2F_ViT_stage(img_size=128, patch_size=[3, 7, 15], stride=[2, 4, 8], num_classes=12,
                      embed_dims=[256, 256, 256], num_heads=[2, 2, 2], mlp_ratios=[2, 2, 2], qkv_bias=False,
                      qk_scale=None, drop_rate=0., attn_drop_rate=0., norm_layer=nn.Identity,
                      depths=[4, 4, 4], sr_ratios=[1, 1, 1], num_stages=3, linear=False).to(device)

# 加載預訓練模型權重
model_path = '../Model/C2FViT_affine_COM_template_matching_tigerdata_RAS/C2FViT_affine_COM_template_matching_tigerdata_RAS_stagelvl3_249000.pth'
print(f"Loading model weight {model_path} ...")
model.load_state_dict(torch.load(model_path))
model.eval()

# 定義轉換器
affine_transform = AffineCOMTransform().to(device)
#affine_transform = CustomAffineCOMTransform().to(device)
init_center = Center_of_mass_initial_pairwise()
#init_center = CustomCenter_of_mass_initial_pairwise()

# 將核心模型和轉換器打包成完整模型
full_model = FullModel(model, affine_transform, init_center).to(device)

# 加載固定影像
fixed_path = '../Data/MNI152_T1_1mm_brain_pad_RSP.nii.gz'
fixed_img_nii = nib.load(fixed_path)
fixed_img = fixed_img_nii.get_fdata()

# 確保影像尺寸是 256x256x256
target_shape = (256, 256, 256)
if fixed_img.shape != target_shape:
    fixed_img = pad_to_shape(fixed_img, target_shape)

fixed_img = min_max_norm(fixed_img)
fixed_img = torch.from_numpy(fixed_img).float().to(device).unsqueeze(0).unsqueeze(0)

# Dummy moving image for ONNX conversion
dummy_moving_img = torch.randn(1, 1, 256, 256, 256).to(device)

# 將完整模型轉換成 ONNX 格式
onnx_path = "mprage_affine_v001_train.onnx"
#onnx_path = "mprage_affine_v001_train_custom.onnx"
torch.onnx.export(full_model, 
                  (dummy_moving_img, fixed_img), 
                  onnx_path, 
                  export_params=True, 
                  opset_version=20, 
                  do_constant_folding=True, 
                  input_names=['moving_img', 'fixed_img'], 
                  output_names=['moved', 'affine_matrix', 'init_flow'])

print(f"Full model has been converted to {onnx_path}")


Loading model weight ../Model/C2FViT_affine_COM_template_matching_tigerdata_RAS/C2FViT_affine_COM_template_matching_tigerdata_RAS_stagelvl3_249000.pth ...


/tmp/ipykernel_1367/1937543338.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Full model has been converted to mprage_rigid_v001_train_custom.onnx


In [35]:
#讀取資料並預測
import onnxruntime as ort
import numpy as np
import nibabel as nib
from Functions import min_max_norm, pad_to_shape, reorient_image

    
moving_nii = nib.load('/NFS/PeiMao/dataset/ABIDE_NoAffine/ABIDE_0050424_tbet.nii.gz')
fixed_nii = nib.load('/NFS/PeiMao/GitHub/C2FViT_Medical_Image/Data/MNI152_T1_1mm_brain_pad_RSP_RAS.nii.gz')
fixed_affine = fixed_nii.affine
fixed_header = fixed_nii.header
moving_nii = reorient_image(moving_nii, ('R', 'A', 'S'))
moving_data = moving_nii.get_fdata().astype(np.float32)
fixed_data = fixed_nii.get_fdata().astype(np.float32)
moving_data = pad_to_shape(moving_data, (256, 256, 256))

fixed_data = np.clip(fixed_data, a_min=2500, a_max=np.max(fixed_data))

# 在第0轴和第1軸位置添加新维度（增加一个 batch size 维度）
moving = np.expand_dims(moving_data, axis=0)
moving = np.expand_dims(moving, axis=1)
fixed = np.expand_dims(fixed_data, axis=0)
fixed = np.expand_dims(fixed, axis=1)

moving = min_max_norm(moving)
fixed = min_max_norm(fixed)

# 创建 ONNX Runtime 会话
session = ort.InferenceSession("mprage_affine_v001_train.onnx")

# 获取输入的名称
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]

# 创建输入字典
inputs = {input_names[0]: moving, input_names[1]: fixed}

# 运行模型推理
outputs = session.run(None, inputs)

# 打印输出结果
print(outputs[0].shape)
print(outputs[1].shape)

# 使用 squeeze 移除长度为 1 的维度
moved = np.squeeze(outputs[0])
affine_matrix = np.squeeze(outputs[1])
init_flow = outputs[2]
print(affine_matrix)
print(init_flow.shape)

# 创建一个 NIfTI 图像对象
moved_nii = nib.Nifti1Image(moved, fixed_affine)

# 保存 NIfTI 图像为 .nii.gz 文件
#nib.save(moved_nii, 'onnx_output_image_rigid.nii.gz')
nib.save(moved_nii, 'onnx_output_image_aff.nii.gz')

(1, 1, 256, 256, 256)
(1, 3, 4)
[[ 0.8344236   0.24871936  0.01360709  0.01237769]
 [-0.18627378  0.9458701  -0.00467041 -0.01208516]
 [-0.014078    0.03331293  0.71046937  0.00340794]]
(1, 256, 256, 256, 3)


In [23]:
# #model轉onnx(C2FViT)(affine transform)
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import nibabel as nib
# import numpy as np
# from C2FViT_model import AffineCOMTransform, Center_of_mass_initial_pairwise
# from Functions import min_max_norm, pad_to_shape

# class FullModel(nn.Module):
#     def __init__(self, init_center):
#         super(FullModel, self).__init__()
#         self.init_center = init_center

#     def forward(self, moving_img, fixed_img, moving_seg, affine_matrix):
#         # Center of mass initialization
#         moving_img, init_flow = self.init_center(moving_img, fixed_img)
        
#         # Downsample the images (not needed for segmentation transformation)
#         # X_down = F.interpolate(moving_img, scale_factor=0.5, mode="trilinear", align_corners=True)
#         # Y_down = F.interpolate(fixed_img, scale_factor=0.5, mode="trilinear", align_corners=True)
        
#         # Apply the grid sample for center of mass alignment
#         moving_seg = F.grid_sample(moving_seg, init_flow, mode='nearest', align_corners=True)
        
#         # Generate the affine grid and apply the affine transformation
#         F_X_Y = F.affine_grid(affine_matrix, moving_seg.shape, align_corners=True)
#         moving_seg = F.grid_sample(moving_seg, F_X_Y, mode='nearest', align_corners=True)  # Use 'nearest' for segmentation
        
#         return moving_seg

# # 設定裝置
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # 定義轉換器
# init_center = Center_of_mass_initial_pairwise()

# # 將轉換器打包成完整模型
# full_model = FullModel(init_center).to(device)

# # 加載固定影像
# fixed_path = '../Data/MNI152_T1_1mm_brain_pad_RSP_RAS.nii.gz'
# fixed_img_nii = nib.load(fixed_path)
# fixed_img = fixed_img_nii.get_fdata()

# # 確保影像尺寸是 256x256x256
# target_shape = (256, 256, 256)
# if fixed_img.shape != target_shape:
#     fixed_img = pad_to_shape(fixed_img, target_shape)

# fixed_img = min_max_norm(fixed_img)
# fixed_img = torch.from_numpy(fixed_img).float().to(device).unsqueeze(0).unsqueeze(0)

# # Dummy moving image, segmentation, and affine matrix for ONNX conversion
# dummy_moving_img = torch.randn(1, 1, 256, 256, 256).to(device)
# dummy_moving_seg = torch.randint(0, 2, (1, 1, 256, 256, 256)).float().to(device)  # Assuming binary segmentation
# dummy_affine_matrix = torch.tensor([[[0.8344236, 0.24871936, 0.01360709, 0.01237769],
#                                      [-0.18627378, 0.9458701, -0.00467041, -0.01208516],
#                                      [-0.014078, 0.03331293, 0.71046937, 0.00340794]]]).to(device)  # Shape (1, 3, 4)

# # 將完整模型轉換成 ONNX 格式
# onnx_path = "mprage_affine_transform_v001_train.onnx"
# torch.onnx.export(full_model, 
#                   (dummy_moving_img, fixed_img, dummy_moving_seg, dummy_affine_matrix), 
#                   onnx_path, 
#                   export_params=True, 
#                   opset_version=20, 
#                   do_constant_folding=True, 
#                   input_names=['moving_img', 'fixed_img', 'moving_seg', 'affine_matrix'], 
#                   output_names=['moved_seg'])

# print(f"Full model has been converted to {onnx_path}")



Full model has been converted to mprage_affine_transform_v001_train.onnx


In [29]:
# #讀取資料並預測(C2FViT)(affine transform)
# import onnxruntime as ort
# import numpy as np
# import nibabel as nib
# from Functions import min_max_norm, pad_to_shape, reorient_image

# # 讀取資料
# moving_nii = nib.load('/NFS/PeiMao/dataset/ABIDE_NoAffine/ABIDE_0050424_tbet.nii.gz')
# moving_seg_nii = nib.load('/NFS/PeiMao/dataset/ABIDE_aseg(tigerbx)/ABIDE_0050424_tbet_aseg.nii.gz')
# fixed_nii = nib.load('/NFS/PeiMao/GitHub/C2FViT_Medical_Image/Data/MNI152_T1_1mm_brain_pad_RSP_RAS.nii.gz')
# fixed_affine = fixed_nii.affine
# fixed_header = fixed_nii.header

# # 重定向moving影像的方向（R, A, S）
# moving_nii = reorient_image(moving_nii, ('R', 'A', 'S'))
# moving_data = moving_nii.get_fdata().astype(np.float32)
# moving_seg_nii = reorient_image(moving_seg_nii, ('R', 'A', 'S'))
# moving_seg_data = moving_seg_nii.get_fdata().astype(np.float32)
# fixed_data = fixed_nii.get_fdata().astype(np.float32)

# # 填充形狀至 256x256x256
# moving_data = pad_to_shape(moving_data, (256, 256, 256))
# moving_seg_data = pad_to_shape(moving_seg_data, (256, 256, 256))

# # 固定影像的值範圍限制在一定範圍內
# fixed_data = np.clip(fixed_data, a_min=2500, a_max=np.max(fixed_data))
# #fixed_data = pad_to_shape(fixed_data, (256, 256, 256))

# # 添加 batch 和 channel 維度
# moving = np.expand_dims(np.expand_dims(moving_data, axis=0), axis=1)
# moving_seg = np.expand_dims(np.expand_dims(moving_seg_data, axis=0), axis=1)
# fixed = np.expand_dims(np.expand_dims(fixed_data, axis=0), axis=1)

# # 正規化
# moving = min_max_norm(moving)
# fixed = min_max_norm(fixed)

# # 創建 ONNX Runtime 會話
# session = ort.InferenceSession("mprage_affine_transform_v001_train.onnx")

# # 獲取輸入和輸出名稱
# input_names = [input.name for input in session.get_inputs()]
# output_names = [output.name for output in session.get_outputs()]

# # 定義仿射矩陣
# affine_matrix = [[[0.8344236, 0.24871936, 0.01360709, 0.01237769],
#                                      [-0.18627378, 0.9458701, -0.00467041, -0.01208516],
#                                      [-0.014078, 0.03331293, 0.71046937, 0.00340794]]]


# # 構建輸入字典
# inputs = {
#     input_names[0]: moving, 
#     input_names[1]: fixed, 
#     input_names[2]: moving_seg, 
#     input_names[3]: np.array(affine_matrix, dtype=np.float32)
# }

# # 运行模型推理
# outputs = session.run(None, inputs)

# # 輸出結果
# moved_seg = outputs[0]

# # 打印输出形状
# print(f"Moved Segmentation Shape: {moved_seg.shape}")

# # 使用 squeeze 移除长度为 1 的维度
# moved_seg = np.squeeze(moved_seg)

# # 創建一個 NIfTI 圖像對象
# moved_seg_nii = nib.Nifti1Image(moved_seg, fixed_affine)

# # 保存 NIfTI 圖像為 .nii.gz 文件
# nib.save(moved_seg_nii, 'onnx_output_seg_test.nii.gz')


Moved Segmentation Shape: (1, 1, 256, 256, 256)


In [3]:
def custom_grid_sample(im, grid, align_corners=False):
    n, c, d, h, w = im.shape

    gn, gd, gh, gw, _ = grid.shape

    assert n == gn

    x = grid[:, :, :, :, 0]
    y = grid[:, :, :, :, 1]
    z = grid[:, :, :, :, 2]

    if align_corners:
        x = ((x + 1) / 2) * (w - 1)
        y = ((y + 1) / 2) * (h - 1)
        z = ((z + 1) / 2) * (d - 1)
    else:
        x = ((x + 1) * w - 1) / 2
        y = ((y + 1) * h - 1) / 2
        z = ((z + 1) * d - 1) / 2

    x = x.view(n, -1)
    y = y.view(n, -1)
    z = z.view(n, -1)

    x0 = torch.floor(x).long()
    y0 = torch.floor(y).long()
    z0 = torch.floor(z).long()
    x1 = x0 + 1
    y1 = y0 + 1
    z1 = z0 + 1

    wa = ((x1 - x) * (y1 - y) * (z1 - z)).unsqueeze(1)
    wb = ((x1 - x) * (y - y0) * (z1 - z)).unsqueeze(1)
    wc = ((x - x0) * (y1 - y) * (z1 - z)).unsqueeze(1)
    wd = ((x - x0) * (y - y0) * (z1 - z)).unsqueeze(1)
    we = ((x1 - x) * (y1 - y) * (z - z0)).unsqueeze(1)
    wf = ((x1 - x) * (y - y0) * (z - z0)).unsqueeze(1)
    wg = ((x - x0) * (y1 - y) * (z - z0)).unsqueeze(1)
    wh = ((x - x0) * (y - y0) * (z - z0)).unsqueeze(1)

    # Apply default for grid_sample function zero padding
    im_padded = F.pad(im, pad=[1, 1, 1, 1, 1, 1], mode='constant')
    padded_d = d + 2
    padded_h = h + 2
    padded_w = w + 2
    # save points positions after padding
    x0, x1, y0, y1, z0, z1 = x0 + 1, x1 + 1, y0 + 1, y1 + 1, z0 + 1, z1 + 1

    # Clip coordinates to padded image size
    device = im.device
    x0 = torch.clamp(x0, 0, padded_w - 1)
    x1 = torch.clamp(x1, 0, padded_w - 1)
    y0 = torch.clamp(y0, 0, padded_h - 1)
    y1 = torch.clamp(y1, 0, padded_h - 1)
    z0 = torch.clamp(z0, 0, padded_d - 1)
    z1 = torch.clamp(z1, 0, padded_d - 1)

    im_padded = im_padded.view(n, c, -1)

    x0_y0_z0 = (x0 + y0 * padded_w + z0 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)
    x0_y1_z0 = (x0 + y1 * padded_w + z0 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)
    x1_y0_z0 = (x1 + y0 * padded_w + z0 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)
    x1_y1_z0 = (x1 + y1 * padded_w + z0 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)
    x0_y0_z1 = (x0 + y0 * padded_w + z1 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)
    x0_y1_z1 = (x0 + y1 * padded_w + z1 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)
    x1_y0_z1 = (x1 + y0 * padded_w + z1 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)
    x1_y1_z1 = (x1 + y1 * padded_w + z1 * padded_w * padded_h).unsqueeze(1).expand(-1, c, -1)

    Ia = torch.gather(im_padded, 2, x0_y0_z0)
    Ib = torch.gather(im_padded, 2, x0_y1_z0)
    Ic = torch.gather(im_padded, 2, x1_y0_z0)
    Id = torch.gather(im_padded, 2, x1_y1_z0)
    Ie = torch.gather(im_padded, 2, x0_y0_z1)
    If_ = torch.gather(im_padded, 2, x0_y1_z1)
    Ig = torch.gather(im_padded, 2, x1_y0_z1)
    Ih = torch.gather(im_padded, 2, x1_y1_z1)

    return (Ia * wa + Ib * wb + Ic * wc + Id * wd + Ie * we + If_ * wf + Ig * wg + Ih * wh).reshape(n, c, gd, gh, gw)

#SPM
def custom_nearest_grid_sample(im, grid, align_corners=False):
    n, c, d, h, w = im.shape

    gn, gd, gh, gw, _ = grid.shape

    assert n == gn

    x = grid[:, :, :, :, 0]
    y = grid[:, :, :, :, 1]
    z = grid[:, :, :, :, 2]

    if align_corners:
        x = ((x + 1) / 2) * (w - 1)
        y = ((y + 1) / 2) * (h - 1)
        z = ((z + 1) / 2) * (d - 1)
    else:
        x = ((x + 1) * w - 1) / 2
        y = ((y + 1) * h - 1) / 2
        z = ((z + 1) * d - 1) / 2

    x = torch.round(x).long()
    y = torch.round(y).long()
    z = torch.round(z).long()

    # Clip coordinates to image size
    x = torch.clamp(x, 0, w - 1)
    y = torch.clamp(y, 0, h - 1)
    z = torch.clamp(z, 0, d - 1)
    #print("im.shape", im[:, :, z, y, x].shape)
    return im[:, :, z, y, x][0]

In [4]:
#model轉onnx(C2FViT)(affine transform)(要輸入init_flow)(nearest)
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
import numpy as np
from C2FViT_model import AffineCOMTransform, Center_of_mass_initial_pairwise
from Functions import min_max_norm, pad_to_shape

class FullModel(nn.Module):
    def __init__(self, init_center):
        super(FullModel, self).__init__()

    def forward(self, moving_seg, init_flow, affine_matrix):
        # Apply the grid sample for center of mass alignment
        moving_seg = F.grid_sample(moving_seg, init_flow, mode='nearest', align_corners=True)
        #moving_seg = custom_nearest_grid_sample(moving_seg, init_flow, align_corners=True)
        
        # Generate the affine grid and apply the affine transformation
        F_X_Y = F.affine_grid(affine_matrix, moving_seg.shape, align_corners=True)
        moving_seg = F.grid_sample(moving_seg, F_X_Y, mode='nearest', align_corners=True)
        #moving_seg = custom_nearest_grid_sample(moving_seg, F_X_Y, align_corners=True)
        
        return moving_seg

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義轉換器
init_center = Center_of_mass_initial_pairwise()
#init_center = CustomCenter_of_mass_initial_pairwise()

# 將轉換器打包成完整模型
full_model = FullModel(init_center).to(device)

#Segmentation, and affine matrix for ONNX conversion
dummy_moving_seg = torch.randint(0, 2, (1, 1, 256, 256, 256)).float().to(device)  # Assuming binary segmentation
dummy_init_flow = torch.randn(1, 256, 256, 256, 3).float().to(device)
dummy_affine_matrix = torch.tensor([[[0.8344236, 0.24871936, 0.01360709, 0.01237769],
                                     [-0.18627378, 0.9458701, -0.00467041, -0.01208516],
                                     [-0.014078, 0.03331293, 0.71046937, 0.00340794]]]).to(device)  # Shape (1, 3, 4)

# 將完整模型轉換成 ONNX 格式
onnx_path = "mprage_affine_transform_v001_train.onnx"
torch.onnx.export(full_model, 
                  (dummy_moving_seg, dummy_init_flow, dummy_affine_matrix), 
                  onnx_path, 
                  export_params=True, 
                  opset_version=20, 
                  do_constant_folding=True, 
                  input_names=['moving_seg', 'init_flow', 'affine_matrix'], 
                  output_names=['moved_seg'])

print(f"Full model has been converted to {onnx_path}")

Full model has been converted to mprage_affine_transform_v001_train_custom.onnx


/tmp/ipykernel_1367/3976391069.py:87: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert n == gn
/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:5645: UserWarning: Exporting aten::index operator of advanced indexing in opset 20 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


In [5]:
#model轉onnx(C2FViT)(affine transform)(要輸入init_flow)(bilinear)
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
import numpy as np
from C2FViT_model import AffineCOMTransform, Center_of_mass_initial_pairwise
from Functions import min_max_norm, pad_to_shape

class FullModel(nn.Module):
    def __init__(self, init_center):
        super(FullModel, self).__init__()

    def forward(self, moving_seg, init_flow, affine_matrix):
        # Apply the grid sample for center of mass alignment
        moving_seg = F.grid_sample(moving_seg, init_flow, mode='bilinear', align_corners=True)
        #moving_seg = custom_grid_sample(moving_seg, init_flow, align_corners=True)
        
        # Generate the affine grid and apply the affine transformation
        F_X_Y = F.affine_grid(affine_matrix, moving_seg.shape, align_corners=True)
        moving_seg = F.grid_sample(moving_seg, F_X_Y, mode='bilinear', align_corners=True)
        #moving_seg = custom_grid_sample(moving_seg, F_X_Y, align_corners=True)
        
        return moving_seg

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義轉換器
init_center = Center_of_mass_initial_pairwise()
init_center = CustomCenter_of_mass_initial_pairwise()

# 將轉換器打包成完整模型
full_model = FullModel(init_center).to(device)

#Segmentation, and affine matrix for ONNX conversion
dummy_moving_seg = torch.randint(0, 2, (1, 1, 256, 256, 256)).float().to(device)  # Assuming binary segmentation
dummy_init_flow = torch.randn(1, 256, 256, 256, 3).float().to(device)
dummy_affine_matrix = torch.tensor([[[0.8344236, 0.24871936, 0.01360709, 0.01237769],
                                     [-0.18627378, 0.9458701, -0.00467041, -0.01208516],
                                     [-0.014078, 0.03331293, 0.71046937, 0.00340794]]]).to(device)  # Shape (1, 3, 4)

# 將完整模型轉換成 ONNX 格式
onnx_path = "mprage_affine_transform_v001_train_bili.onnx"
#onnx_path = "mprage_affine_transform_v001_train_bili_custom.onnx"
torch.onnx.export(full_model, 
                  (dummy_moving_seg, dummy_init_flow, dummy_affine_matrix), 
                  onnx_path, 
                  export_params=True, 
                  opset_version=20, 
                  do_constant_folding=True, 
                  input_names=['moving_seg', 'init_flow', 'affine_matrix'], 
                  output_names=['moved_seg'])

print(f"Full model has been converted to {onnx_path}")

/tmp/ipykernel_1367/3976391069.py:6: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert n == gn


Full model has been converted to mprage_affine_transform_v001_train_bili_custom.onnx
